In [1]:
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
import time
requests.adapters.DEFAULT_RETERIES = 5
import os

# Make dir for storing

In [ ]:
#好看的创建目录
path = "./book"
isExists = os.path.exists(path)
# 判断结果
if not isExists:
# 如果不存在则创建目录,创建目录操作函数
    os.makedirs(path)
    print(path + ': Created successfully')
else:
    # 如果目录存在则不创建，并提示目录已存在
    print(path + ': Directory already exists')

# First version of crawling

In [ ]:
# orignial part
ID_list=[]
author_list=[]
book_n_list=[]
text_list=[]
current=3400
error_time=0
ID=1
headers = {'User-Agent':'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
               ' (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
while(ID<=1000):
    sign_allow=1
    current+=2
    url = 'https://www.gutenberg.org/ebook/'+str(current)
    
    r = requests.get(url, headers=headers)
    # beautiful soup
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, "html.parser")
    # getting table
    try:
        table = html_soup.find('table',class_="bibrec")
        rows = table.find_all('tr')
        author_all=[]
        for row in rows:
            title=row.find('th')
            try:
                title_name=title.text
                if(title_name=="Author"):
                    author_loc = row.find("a")
                    str_list=(author_loc.text).split(",")
                    for author in str_list[:-1]:
                        author_all.append(author)
            except:
                pass

            try:
                title_name=title.text
                if(title_name=="Title"):
                    book_name = row.find("td")
                    b_title=book_name.text
                    b_title=b_title.replace("\n","")
            except:
                pass

            try:
                title_name=title.text
                if(title_name=="Language"):
                    L_type = row.find("td")
                    if(L_type.text!="English" and L_type.text!="英语"):
                        print("Detect "+L_type.text+" for "+str(current)+". Skip to the next")
                        sign_allow=0
                        break
            except:
                pass


        if(sign_allow!=1):
            continue

        if(len(author_all)==0):
            print("Get the book "+str(current)+" fail since no author. Go to the next one")
            continue
        if(len(b_title)==0):
            print("Get the book "+str(current)+" fail since no title. Go to the next one")
            continue

        plain_text_url=url+".txt.utf-8"
        try:
            urllib.request.urlretrieve(plain_text_url,'./book/'+str(ID)+'.txt')
        except:
            print("Download the file fail")
            continue


        fp=open('./book/'+str(ID)+'.txt', 'r', encoding='utf-8')
        ID_list.append(ID)
        author_list.append(author_all)
        book_n_list.append(b_title)
        text_list.append(fp.read())
        fp.close()
        print("Finish the "+str(ID)+" book")

        ID+=1

        if(ID%50==0):
            result = pd.DataFrame({'ID': ID_list, 'Author': author_list, 'Book_name': book_n_list, 'text': text_list})
            # 写入dataframe，不带unnamed
            #display(result)
            print("Start to write,the coding format is UTF-8")
            result.to_csv('book_final.csv', index=False)
            print("write successfully")
            time.sleep(1)
            break
    except:
        if(error_time%2==0):
            error_time+=1
            headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
        elif(error_time%2==1):
            error_time+=1
            headers = {'User-Agent':'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
               ' (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36','Cookie': 'session_id=1b1f4756d9d79803e06391a7699920ee686e75f8'}
        else:
            pass
        print("Error, and sleep for 3 second")
        time.sleep(3)
        

result = pd.DataFrame({'ID': ID_list, 'Author': author_list, 'Book_name': book_n_list, 'text': text_list})
# 写入dataframe，不带unnamed
display(result)
print("Start to write,the coding format is UTF-8")
result.to_csv('book_final.csv', index=False)
print("write successfully")

# Test whether the web page can be scraping

In [ ]:
# check the above page if it is ban 
#headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
url='https://www.gutenberg.org/ebooks/3330'
r = requests.get(url)#,headers=headers)
r.text

# building the IP pool

In [2]:
# grab proxy
import random
page=999
url = 'http://www.xiladaili.com/gaoni/'

headers = {'User-Agent':'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
               ' (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36','Cookie': 'session_id=1b1f4756d9d79803e06391a7699920ee686e75f8'}
http_proxy = []
num=0
while num<64:
    page+=1
    current_url=url+str(page)
    page_text = requests.get(url, headers=headers).text
    html_soup = BeautifulSoup(page_text, "html.parser")
    tables = html_soup.find("tbody")
    lines = tables.find_all("tr")

    for line in lines:
        each=line.find_all("td")
        if each[1].text=="HTTPS代理" or each[1].text=="HTTP,HTTPS代理":
            dic = {'https': each[0].text}
            http_proxy.append(dic)
            num+=1
print(len(http_proxy))
#print(http_proxy)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
http_proxy

# Testing IP pool to crawl

In [ ]:
# check random select
import random
url='https://www.gutenberg.org/ebooks/1555'
r = requests.get(url,headers=headers,proxies=random.choice(http_proxy)).text
r

# change another site (successful version)

In [ ]:
# test for whole text
current=555
url='https://www.gutenberg.org/cache/epub/'+str(current)+'/pg'+str(current)+'.txt'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
r = requests.get(url, headers=headers)
# beautiful soup
html_contents = r.text
print(html_contents)

In [ ]:
# final successful version
import re
#display(plain_text)

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
requests.adapters.DEFAULT_RETERIES = 5
ID=450
current=4000
ID_list=[]
author_list=[]
book_n_list=[]
text_list=[]
while ID<600:
    try:
        error_count=0
        url='https://www.gutenberg.org/cache/epub/'+str(current)+'/pg'+str(current)+'.txt'
        r = requests.get(url, headers=headers,proxies=random.choice(http_proxy))
        # beautiful soup
        plain_text = r.text
        #print(plain_text)
        current+=1

        if(len(re.findall(r"<title>(.+?) ",plain_text))== 0 and re.findall(r"Language: (.+?)\r",plain_text))[0]=="English":
            with open('./book/' + str(ID) + '.txt', "w", encoding='utf-8') as f:
                f.write(plain_text)
            f.close()
            author_all=re.findall(r"Author: (.+?)\r",plain_text)

            b_title=re.findall(r"Title: (.+?)\r",plain_text)

            print("Finish the book of "+str(ID))
            ID_list.append(ID)
            author_list.append(author_all)
            book_n_list.append(b_title[0])
            text_list.append(plain_text)
            ID+=1
        else:
            pass
    except:
        print('error')
    
    if (ID % 50 == 0):
        result = pd.DataFrame({'ID': ID_list, 'Author': author_list, 'Book_name': book_n_list, 'text': text_list})
        # 写入dataframe，不带unnamed
        # display(result)
        print("Start to write,the coding format is UTF-8")
        result.to_csv('book_final.csv', index=False)
        print("write successfully")
        time.sleep(1)
result = pd.DataFrame({'ID': ID_list, 'Author': author_list, 'Book_name': book_n_list, 'text': text_list})
# 写入dataframe，不带unnamed
# display(result)
print("Start to write,the coding format is UTF-8")
result.to_csv('book_final.csv', index=False)
print("write successfully")
time.sleep(1)

## Change the encoding format and name 

In [5]:
import pandas as pd
df=pd.read_csv("book_final.csv")
for i in range(df.shape[0]):
    process=df.iloc[i, 2]
    after_p=process.split("\r")
    df.iloc[i, 2]=after_p[0]
df.to_csv('book_final.csv',index=False)

In [6]:
df.iloc[2231,2]

"Hunter Quatermain's Story"

In [7]:
# changing the name of the txt file
import os
count_at=0
count_low=0
replace_list=[ '\\' ,"/" ,':',"?", "#" ,'"','<','>',"|"]
for index,row in df.iterrows():  
    name=str(row['Book_name'])
#     try:
    #print(index+1,row['Author'],row['Book_name']) 
    string=name+"_"
    for i in eval(row['Author']):
        string+=str(i)[1:]+"&"
    string=string[:-1]
    string+=".txt"
    #print(string)
    ori=str(index+1)+".txt"
    new=string
    for each in replace_list:
        new=new.replace(each," ")
#     print(index)
    error_time=0
    neww=new
    while(1):
        try:
            os.rename("C:\\Users\\Lenovo\\jupyter\\cralw\\book\\"+ori,"C:\\Users\\Lenovo\\jupyter\\cralw\\book\\"+neww)
            print("index done")
            break
        except:
            error_time+=1
            neww=new[:-4]+" V"+str(error_time)+".txt"
            print(neww)
            continue

index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done

The History of The Decline and Fall of the Roman Empire_dward Gibbon V1.txt
index done
The History of The Decline and Fall of the Roman Empire_dward Gibbon V1.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V2.txt
index done
The History of The Decline and Fall of the Roman Empire_dward Gibbon V1.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V2.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V3.txt
index done
The History of The Decline and Fall of the Roman Empire_dward Gibbon V1.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V2.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V3.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V4.txt
index done
The History of The Decline and Fall of the Roman Empire_dward Gibbon V1.txt
The History of The Decline and Fall of the Roman Empire_dward Gibbon V2.txt
The History of The Decline and Fall of the R

index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done

index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
An Account of Egypt_erodotus V1.txt
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
The Works of Edgar Allan Poe_dgar Allan Poe V1.txt
index done
The Works of Edgar Allan Poe_dgar Allan Poe V1.txt
The Works of Edgar Allan Poe_dgar Allan Poe V2.txt
index done
The Works of Edgar Allan Poe_dgar Allan Poe V1.txt
The Works of Edgar Allan Poe_dgar Allan Poe V2.txt
The Works of Edgar Allan Poe_dgar Allan Poe V3.txt
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index don

index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done
index done

In [ ]:
# testing
import os
ori=str(index+1)+".txt"
new=string
os.rename("C:\\Users\\Lenovo\\jupyter\\cralw\\","C:\\Users\\Lenovo\\jupyter\\cralw\\book name.txt")
print("重命名"+str(index+1))

In [ ]:
new="Andrea Delfin_ Paul Heys.txt"
attribute=new.split("_")
book_name=attribute[0]
author_list=[]
for each in attribute[1][:-4].split(" "):
    if(each!=""):
        author_list.append(each)
print(author_list)
print(book_name)

In [ ]:
new="The Twin Hells\rA Thrilling Narrative of Life in the Kansas and Missouri Penitentiaries"
replace_list=[ '\\' ,"/" ,':',"?", "#" ,'"','<','>',"|","\r"]
for each in replace_list:
    new=new.replace(each," ")
    print(each)
new

In [ ]:
process=df.iloc[4, 2]
after_p=process.split("\r")
df.iloc[4, 2]=after_p[0]
df

In [ ]:
for i in range(df.shape[0]):
    process=df.iloc[i, 2]
    after_p=process.split("\r")
    df.iloc[i, 2]=after_p[0]
df.to_csv('book_final.csv')
df.shape[0]

# changing the encoding format

In [ ]:
import os
from os import listdir
from chardet import detect

fileList=os.listdir("C:\\Users\\Lenovo\\jupyter\\cralw\\book\\")

fns = (fn for fn in fileList if fn.endswith('.txt'))
count=0
for fn in fns:
    with open("C:\\Users\\Lenovo\\jupyter\\cralw\\book\\"+fn, 'rb+') as fp:
        content = fp.read()
        encoding = detect(content)['encoding']
        content = content.decode(encoding).encode('utf8')
        fp.seek(0)
        fp.write(content)
    print("finish"+str(count))
    count+=1